In [1]:
import geopandas as gpd
import pandas as pd
import boto3

In [2]:
bucket_name = 'opera-validation-datasets'
root_folder_name = 'calval-database-dev/'
upload_files_path = './upload/'

In [3]:
filename = '20211011_155455_52_2262_3B_AnalyticMS_SR_8b_clip_Supervised_MD.tif'
logfile = 'logfile.txt'
training_data = 'trainingInput1.scp'
source_image_name = '20211011_155455_52_2262'
calc_type = 'Classification'
final = False
fitness = 0
public = True

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [5]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')

In [6]:
search = collocation[(collocation.pl_ID == source_image_name)].sort_values('hls_pl_AOI_intersect',ascending=False)
search = search.iloc[[0]]
site = search.chip_ID.iloc[0]

In [7]:
image_calc = search[['pl_ID','geometry']].rename(columns={'pl_ID':'image_name'})
image_calc.head()

,image_name,geometry
39186,20211011_155455_52_2262,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [8]:
image_calc['image_calc_name'] = filename[:-4]
image_calc['calc_type'] = calc_type
image_calc['final'] = final
image_calc['fitness'] = fitness
image_calc['public'] = public
image_calc.head()

,image_name,geometry,image_calc_name,calc_type,final,fitness,public
39186,20211011_155455_52_2262,"POLYGON ((-90.50851 16.35841, -90.54697 16.176...",20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,Classification,False,0,True


In [9]:
s3_folder_path = root_folder_name+'data/'+site+'/'+source_image_name+'/'+filename[:-4]+'/'
s3_link_image = s3_folder_path+filename
s3_link_logfile = s3_folder_path+logfile
s3_link_training = s3_folder_path+training_data

In [10]:
image_calc['s3_link_image_calc'] = s3_link_image
image_calc['s3_link_logfile'] = s3_link_logfile
image_calc['s3_link_training'] = s3_link_training

In [11]:
response = s3_client.upload_file(upload_files_path+filename,bucket_name,s3_link_image)
response = s3_client.upload_file(upload_files_path+logfile,bucket_name,s3_link_logfile)
response = s3_client.upload_file(upload_files_path+training_data,bucket_name,s3_link_training)

In [12]:
image_calc_bytes = bytes(image_calc.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,root_folder_name+'image_calc.geojson')
s3object.put(Body=image_calc_bytes)

{'ResponseMetadata': {'RequestId': 'RX2A0HK3HRK8XNPK',
  'HostId': '3F/J3mlI5+ZIf0rtODoa+O4lrWn3wEjD94mtuPbjkBXhsamQU2hGmn/MP14oFx8NOqT79EhRgQE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3F/J3mlI5+ZIf0rtODoa+O4lrWn3wEjD94mtuPbjkBXhsamQU2hGmn/MP14oFx8NOqT79EhRgQE=',
   'x-amz-request-id': 'RX2A0HK3HRK8XNPK',
   'date': 'Tue, 21 Jun 2022 16:52:25 GMT',
   'etag': '"f7af1ab300cb453cdfc3fe53ff130805"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f7af1ab300cb453cdfc3fe53ff130805"'}